In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import logging

import scipy.stats
import numpy as np

import diffxpy.api as de

/Users/david.fischer/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Generate some data:

In [ ]:
from batchglm.api.models.nb_glm import Simulator

sim = Simulator(num_observations=2000, num_features=100)
sim.generate_sample_description(num_batches=2, num_confounders=2)
# sample parameters from positive truncated N(1, 0.1)
mu=1; phi=0.1
sim.generate_params(rand_fn_loc=lambda size: mu + scipy.stats.truncnorm.rvs(-mu / phi, np.infty, scale=phi, size=size))
sim.generate_data()

# count data
X = sim.X
# sample description
sample_description = sim.sample_description

The sample description should be a pandas DataFrame with `num_observations` rows.
Each column should represent a property of the dataset.

The module `batchglm.api.data` contains some helper functions which can be useful to create this sample description:

- `sample_description_from_anndata()`
- `sample_description_from_xarray()`

In [ ]:
sample_description.drop_duplicates()

# Run differential expression test:

Here, we run a t-test between the condition groups within each batches. Similarly, all other simple differential expression tests can be mapped onto a partition of the data set.

In [ ]:
logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("batchglm").setLevel(logging.INFO)
logging.getLogger("diffxpy").setLevel(logging.INFO)

import diffxpy.api as de

# Create partitions of data set.
part = de.test.partition(
    data=X, 
    partition = "batch",
    sample_description=sample_description)
# Run tests on each partition.
test = part.t_test(
    grouping="condition"
)

# Obtaining the results

## Obtain results across all tests

In [ ]:
test.summary().iloc[:10,:]

## Obtain results for one specific test

The individual partitions are:

In [ ]:
test.partitions

In the following, we will inspect the test within partition '0'.

The p-/q-values can be obtained by calling test.pval / test.qval:

In [ ]:
test.tests[0].qval[:10]

test.summary() returns a pandas DataFrame with a quick overview of the test results:

In [ ]:
test.tests[0].summary().iloc[:10,]

- `gene`: gene name / identifier
- `pval`: p-value of the gene
- `qval`: multiple testing - corrected p-value of the gene
- `log2fc`: log_2 fold change between `no coefficient` and `coefficient`

`test.plot_volcano()` creates a volcano plot of p-values vs. fold-change:

In [ ]:
test.tests[0].plot_volcano()